In [1]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score
from multiprocessing import Pool
import scipy as sc
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns =100
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import gensim
import pickle
from gensim.models import Word2Vec, FastText, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import sys
sys.path.insert(0, '..')
from Common import preprocessing,evaluation,ner,CosineClassifier as cos
classes_map = {'DOC':0, 'ENTER':1, 'ORG':2, 'PRIV':3, 'RANG':4, 'HOST':5}

/home/alex/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alex/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('..//Data//data.txt', delimiter=';', engine='python',encoding='utf8')

In [3]:
questions = np.array(df.question)
questions = preprocessing.preprocess_list(questions)

Здравствуйте, подскажите пожалуйста, возможно ли и законно ли поступление на бюджет в УНИВЕРСИТЕТ, в то время являясь студенткой 3 или 4 курса университета, учась на платном?
Здравствуйте, подскажите пожалуйста, возможно ли и законно ли поступление на бюджет в УНИВЕРСИТЕТ, в то время являясь студенткой 3 или 4 курса УНИВЕРСИТЕТ, учась на платном?
Здравствуйте. заинтересовались вашим УНИВЕРСИТЕТ. Список предоставил военный комиссариат. Ответьте пожалуйста: если я поступаю в ВУЗ ваш ну к примеру Наноэллетроника, далее изъявлю желание учиться на военной кафедре, после успешного окончания  учебы я получаю  две специальности (2 диплома)? Иногородним предоставляется общежитие?  Интересуют еще языки (английский, китайский?)
Здравствуйте. заинтересовались вашим УНИВЕРСИТЕТ. Список предоставил военный комиссариат. Ответьте пожалуйста: если я поступаю в УНИВЕРСИТЕТ ваш ну к примеру Наноэллетроника, далее изъявлю желание учиться на военной кафедре, после успешного окончания  учебы я получаю  две 

Хочу перейти в ваш УНИВЕРСИТЕТ на 2 курс на факультет ЭКТ.Закончил 1 курс ЧГУ без 3.Каковы мои шансы попасть на бюджетное отделение и надо-ли сдавать какие-нибудь экзамены? заранее благодарен за ответ.
Если абитуриент будет рекомендован к зачислению во второй волне, в какой срок необходимо предоставить оригиналы документов? Есть ли шанс при зачислении во второй волне получить место в ОБЩЕЖИТИЕ?
как можно связаться с деканом,если сейчас деканат в отпуске?слышал,что всё равно декан появляется в УНИВЕРСИТЕТ
Здравствуйте, я закончила школу в 2009 году, поступила в УНИВЕРСИТЕТ, но по семейным обстоятельствам на середине зимней сессии 1 курса забрала документы, когда речь зашла об переводе в другое место, сказали первый курс не переводят, в 2013 году мне надо будет сдавать заново сдавать ЕГЭ, смотря в какой вуз я хочу поступить?
Здравствуйте, я закончила школу в 2009 году, поступила в УНИВЕРСИТЕТ, но по семейным обстоятельствам на середине зимней сессии 1 курса забрала документы, когда речь 

Здравствуйте! Прошу прояснить следующую ситуацию: при поступлении на специальности творческой направленности, предусмотрен творческий очный экзамен, который в МИЭТ, например, длится 3 дня (факультет Дизайна). Учащийся, выбрав для поступления профессию дизайнера, будет подавать заявления и в другие УНИВЕРСИТЕТ по данной специальности. И в каждом из них будет творческий конкурс/экзамен, на который тоже надо порядка 2-3 дней. Вопрос: что предусмотрено правилами приёма при совпадении дней сдачи экзаменов творческой направленности в различных ВУЗах? 
Здравствуйте! Прошу прояснить следующую ситуацию: при поступлении на специальности творческой направленности, предусмотрен творческий очный экзамен, который в МИЭТ, например, длится 3 дня (факультет Дизайна). Учащийся, выбрав для поступления профессию дизайнера, будет подавать заявления и в другие УНИВЕРСИТЕТ по данной специальности. И в каждом из них будет творческий конкурс/экзамен, на который тоже надо порядка 2-3 дней. Вопрос: что предусмот

Спасибо Вам за развернутый ответ. Но все равно не совсем понял. Если впереди меня 25 человек с более высокими баллами, а я например по сумме набранных баллов 50, то в данном случае мой оригинал, поданный в УНИВЕРСИТЕТ что даст? Просто я читая информацию по другим ВУЗам понял следующее - читаю приказ о зачислении - попал, то бегу срочно сдавать оригиналы? Я не правильно понимаю ситуацию? Вы меня извините, но трудно мне понять все это.
Спасибо Вам за развернутый ответ. Но все равно не совсем понял. Если впереди меня 25 человек с более высокими баллами, а я например по сумме набранных баллов 50, то в данном случае мой оригинал, поданный в УНИВЕРСИТЕТ что даст? Просто я читая информацию по другим УНИВЕРСИТЕТ понял следующее - читаю приказ о зачислении - попал, то бегу срочно сдавать оригиналы? Я не правильно понимаю ситуацию? Вы меня извините, но трудно мне понять все это.
Здраствуйте. Я закончил ХТКЭМ в 2011г.по специальности Программирование в компьютерных сетях хочу продолжить образован

Здравствуйте.. Я нахожусь в списке рекомендованных к зачислению.(204 балла) будет ли мне предоставлено ОБЩЕЖИТИЕ? Я иногородний. С уважением, Глеб.
Если по баллам не пройду на ОБЩЕЖИТИЕ,есть ли какие-то льготы как социально незащищенной категории граждан.Мать пенсионерка и у нее нас двое иждивенцев.Или все же лучше забрать документы?
Здравствуйте.. Я нахожусь в списке рекомендованных к зачислению.(239 баллов) будет ли мне предоставлено ОБЩЕЖИТИЕ? Я иногородний... и сколько вообще мест с ОБЩЕЖИТИЕм в этом году?
Здравствуйте.. Я нахожусь в списке рекомендованных к зачислению.(239 баллов) будет ли мне предоставлено ОБЩЕЖИТИЕ? Я иногородний... и сколько вообще мест с ОБЩЕЖИТИЕм в этом году?
Здравствуйте, хотел узнать, если ближе  к 4-му августа астанутся места в ОБЩЕЖИТИЕ, есть ли у меня шансы  его получить кол-во баллов 163 факультет ЭТМО орегиналы уже в МИЭТе? Нужно ли мне вторично заполнять заявление №2?  Небудет ли поздно? 
У меня 202 б. Поступаю на ЭКТ. Будет ли место в ОБЩЕЖИТИЕ?
здр

Сколько баллов требуется для предоставления ОБЩЕЖИТИЕ на факультете МПиТК?
А если не получить регистрацию в ОБЩЕЖИТИЕ (не пройти на него конкурс ) ,то надо будет делать прописку в Московской области , Москве , Зеленограде?
Или независимо от прохода на ОБЩЕЖИТИЕ мне там сделают регистрацию?
Здравствуйте! Согласитесь самостоятельно сделать временную регистрацию не легко. Студентам, которым не предоставлено ОБЩЕЖИТИЕ (ПрИТ) надо самим арендовать жилье, и не каждый согласиться предоставить временную регистрацию. В Положении о предоставлении мест в Студгородке ясно сказано, что к 1 сентябрю таких студентов начнут отчислять. Согласитесь это не очень-то приятно. Или я что-то не поняла? 2 недели до посвящения, а мы пока не то-что регистрацию, мы квартиру (комнату) найти не можем. Может с отчислением не так все страшно????
Здравствуйте. Скажите, места в ОБЩЕЖИТИЕ уже заранее для каждого распределены или нужно, так сказать, приехать как можно раньше, чтобы успеть занят какое нибудь место?
Возмож

Добрый день. Дочь поступает в УНИВЕРСИТЕТ в следующем году. Возможна ли подача документов в заочной форме (по обычной или электронной почте)? 
Достаточно результатов ЕГЭ для поступления на ГумФ? 
здравствуйте. мой сын учится в 11 классе, он инвалид-колясочник, собирается поступать к вам по особой квоте на лингвистику, перевод и переводоведение. возможно ли получение высшего образование в Вашем УНИВЕРСИТЕТ с минимальным количеством выездов из дома (так как огромная проблема привезти)? есть ли доступная среда  в корпусах для лиц передвигающихся на колясках? есть ли какие либо дистанционные курсы для поступающих?  спасибо. 
Добрый день. Подскажите пожалуйста, возможно ли перевестись к вам из другого УНИВЕРСИТЕТ? 
Актуальны ли сейчас образцы вступительных экзаменов проводимых УНИВЕРСИТЕТ которые в открытом доступе на сайте?  На все техн. специальности необходима профильная математика, а её пробного нет. 
Здравствуйте, я обучаюсь В УНИВЕРСИТЕТ на аэрокосмическом факультете, очно(4 курс, бюд

Добрый день. На вопрос который был написан про ОБЩЕЖИТИЕ, да студентка первого курса. Заселение в ОБЩЕЖИТИЕ в Лысьвенском филиале будет проходить 31 августа правильно понимаю? 
Добрый день. На вопрос который был написан про ОБЩЕЖИТИЕ, да студентка первого курса. Заселение в ОБЩЕЖИТИЕ в Лысьвенском филиале будет проходить 31 августа правильно понимаю? 
Когда проходить мед осмотр для заселения в ОБЩЕЖИТИЕ? 
Можно ли моему брату заплатить за ОБЩЕЖИТИЕ или нужно лично ? 
Если флюорография пройдена 2017.12.31, то нужно её переделывать или нет? ( Для заселения в ОБЩЕЖИТИЕ) 
Можно узнать время до скольки можно заплатить за ОБЩЕЖИТИЕ в пятницу ? 
Здравствуйте хотела бы уточнить заселение в ОБЩЕЖИТИЕ 31 августа будет? 
Здравствуйте. Скажите,если я закончила GNF колледж по специальности прикладная геодезия. Я могу только на эту специальность поступить или нет? И надо ли сдавать вступительные экзамены? 
Куда подходить за ОБЩЕЖИТИЕ? Ещё не поздно сдать заявление? 
Возьмут ли в ОБЩЕЖИТИЕ без флюрог

Здравствуйте, какой интернет-провайдер в ОБЩЕЖИТИЕ на комплексе? 
А в какой аудитории будет зачисление бюджета ЭТФ факультета 15.08.2018? 
поступила на GNF-нефтяной факультет(бюджет).Нуждаюсь в общежитии. от друзей узнала что направили в 5. можно узнать об этом подробнее, только на месте? заселение в это общежитие когда? 
поступила на GNF-GNF факультет(бюджет).Нуждаюсь в общежитии. от друзей узнала что направили в 5. можно узнать об этом подробнее, только на месте? заселение в это общежитие когда? 
поступила на GNF-GNF факультет(бюджет).Нуждаюсь в ОБЩЕЖИТИЕ. от друзей узнала что направили в 5. можно узнать об этом подробнее, только на месте? заселение в это общежитие когда? 
поступила на GNF-GNF факультет(бюджет).Нуждаюсь в ОБЩЕЖИТИЕ. от друзей узнала что направили в 5. можно узнать об этом подробнее, только на месте? заселение в это ОБЩЕЖИТИЕ когда? 
Поступаю на ЭТФ факультет, программная инженерия, вступительные пройдены успешно. Осенью призовут в РА, могу ли я оформить контракт и ср

Здравствуйте. СТИПЕНДИЯ платятся только тем, кто учится на бюджете? 
во сколько будет зачисление на GNF по специальности мон? 
А что в приоритете при зачислении-результаты внутренних экзаменов или результаты егэ?Читал,что в некоторых УНИВЕРСИТЕТ добавляют некоторое количество бюджетных мест именно для поступающих по егэ.Здесь такое есть? 
в первую волну попал в зачисление,но хотим забрать оригинал. можно забрать оригинал 2-го числа из бф УНИВЕРСИТЕТ? 
до какого числа можно забрать оригинал ,если решил попытаться сдать его в другой УНИВЕРСИТЕТ во вторую волну? 
До скольки сегодня будет работать приемная комиссия GNF? 
Здравствуйте, а как можно отказаться от ОБЩЕЖИТИЕ? 
Нужны ли фотографии, если я подавал на несколько направлений(на каждое по 2) и в сумме у меня их около 4-6 именно в УНИВЕРСИТЕТ? 
Здравствуйте! Я хочу переводиться с тнв (БФ УНИВЕРСИТЕТ) на химическую технологию в головной институт, куда я попаду? На общий профиль или также на тнв?(После первого семестра) 
Здравствуйте! Я

In [74]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
X = vectorizer.fit_transform(questions)

In [75]:
classes = np.array(df['class'])
y = list(map(lambda x: classes_map[x],classes))

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle=True)

In [104]:
log_reg = OneVsRestClassifier(LogisticRegression(random_state=0,C=10,solver='lbfgs',)).fit(X_train, y_train)

In [105]:
print(evaluation.get_CV_scores(log_reg,X_test,y_test,'f1_micro').mean())

0.6951991828396322


In [24]:
count_model = CountVectorizer(min_df=3,ngram_range=(1,1))
X_ = count_model.fit_transform(questions)
Xc = (X_.T * X_)
Xc.setdiag(0)
co_occur = Normalizer().fit_transform(Xc.todense())
print(Xc.todense().shape)

(736, 736)


In [22]:
model = Word2Vec([q.split() for q in questions], size=300, window=5,)
model.train([q.split() for q in questions],epochs=200,total_examples=model.corpus_count)
#model = KeyedVectors.load_word2vec_format("..//..//web_upos_cbow_300_20_2017.bin.gz", binary=True, unicode_errors='ignore')

(1549772, 2718000)

In [23]:
word2vec = []

for q in questions:
    vec = [model.wv[w] for w in q.split() if w in model.wv]
    if len(vec) < 1:
        word2vec.append(np.zeros(300))
    else:
        word2vec.append(np.array(np.sum(vec,axis=0)))

In [99]:
embeddings = []
for q in questions:
    vec = [enth[w][0] for w in q.split() if w in list(enth.keys())]
    if len(vec) < 1:
        embeddings.append(np.zeros(200))
    else:
        embeddings.append(np.array(np.mean(vec,axis=0)))

In [78]:
enthropy = []
for q in questions:
    question = q.split()
    q_dic = Counter(question)
    q_len = len(question)
    vector = []
    cnt=0
    for w in vectorizer.get_feature_names():
            if w in question:
                vector.append(q_dic[w]/q_len*np.log2(q_dic[w]/q_len))
            else:
                vector.append(-0.00001)
            cnt=cnt+1
    enthropy.append(vector.copy())

In [79]:
svd = TruncatedSVD(n_components=200,algorithm='arpack')
matr = svd.fit_transform(np.array(enthropy).T)

In [80]:
matr = Normalizer().fit_transform(matr)

In [81]:
enth = {}
for i in list(zip(vectorizer.get_feature_names(),matr)):
    enth[i[0]]=[i[1]]

In [8]:
pd.DataFrame(word2vec).to_csv("word2vec.tsv", sep='\t',index=False,header=None)

In [59]:
weights = {}
for i in range(len(co_occur)):
    weights[vectorizer.get_feature_names()[i]] = sum(co_occur[i])
    
norm_weights = list(weights.values())/np.linalg.norm(list(weights.values()))
for i in range(len(co_occur)):
    weights[vectorizer.get_feature_names()[i]] = norm_weights[i]


In [29]:
pairs = []
for i in range(matr.shape[0]):
    for j in range(matr.shape[0]):
        if i!=j and vectorizer.get_feature_names()[i] in model.wv and vectorizer.get_feature_names()[j] in model.wv:
            if cosine_similarity([model.wv[vectorizer.get_feature_names()[i]]],[model.wv[vectorizer.get_feature_names()[j]]]) > 0.5:
                if str(i)+' '+str(j) not in pairs and str(j)+' '+str(i) not in pairs:
                    pairs.append(str(i)+' '+str(j))
                    mean = np.mean(np.mean([matr[i],matr[j]],axis=0))
                    matr[i] = mean
                    matr[j] = mean
                    print(vectorizer.get_feature_names()[i],vectorizer.get_feature_names()[j])

абитуриент зелёный
адрес корочка
адрес отправлять
адрес письмо
адрес посылать
адрес точный
адрес эл
английский иностранный
английский интересовать
английский язык
анкета заполнить
анкета ксерокопия
анкета серия
анкета страница
армия действительный
армия летом
армия осень
армия отсрочка
армия пойти
армия призыв
армия сын
асу заочко
асу нгд
асу расписание
асу ускоренный
аудитория заочник
аудитория предоставлять
аудитория присутствовать
аудитория сколька
аэрокосмический одновременно
база вместо
база проживать
бакалавр кафедра
безопасность горный
безопасность информационный
безопасность система
билет собрание
билет студенческий
большой маленький
будущее комплекс
будущее олимпиада
бф пнипа
бюджетник социальный
бюджетник стипендия
бюджетник студенческий
вакантный равный
взять отпуск
вид почта
вид эл
вид электронный
видеть выложить
внутренний действительный
военкомат новый
военный кафедра
военный служба
восстановиться гнф
восстановиться осень
восстановиться очно
восстановиться перейти
временн

KeyboardInterrupt: 